In [1]:
import pandas as pd
import pickle
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

In [2]:
df = pd.read_csv('anime.csv')
df = df.dropna()
df['description'] = df['name'] +' '+ df['genre'] + ' ' +df['type']+' episodes: '+ df['episodes']

In [10]:
df['description']

0        Kimi no Na wa. Drama, Romance, School, Superna...
1        Fullmetal Alchemist: Brotherhood Action, Adven...
2        Gintama° Action, Comedy, Historical, Parody, S...
3             Steins;Gate Sci-Fi, Thriller TV episodes: 24
4        Gintama&#039; Action, Comedy, Historical, Paro...
                               ...                        
12289    Toushindai My Lover: Minami tai Mecha-Minami H...
12290                   Under World Hentai OVA episodes: 1
12291    Violence Gekiga David no Hoshi Hentai OVA epis...
12292    Violence Gekiga Shin David no Hoshi: Inma Dens...
12293    Yasuji no Pornorama: Yacchimae!! Hentai Movie ...
Name: description, Length: 12017, dtype: object

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

anime_embeddings = model.encode(df['description'].tolist())

c:\Users\RS\miniconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\RS\miniconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [7]:
from sklearn.metrics.pairwise import cosine_similarity

def get_recommendations(query, embeddings, df, top_n=5):
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, embeddings)
    top_indices = similarities[0].argsort()[-top_n:][::-1]
    return df.iloc[top_indices]

In [8]:
query = "Funny anime I can watch with friends"
recommendations = get_recommendations(query, anime_embeddings, df)
print(recommendations[['name', 'genre']])

                                          name  \
7363  Sentou Yousei Shoujo Tasukete! Mave-chan   
8140            Anime TV de Hakken! Tamagotchi   
4294      SKET Dance: SD Character Flash Anime   
1061                        Isshuukan Friends.   
2850                       Oshiete! Galko-chan   

                                             genre  
7363  Comedy, Parody, Sci-Fi, Shounen, Super Power  
8140          Comedy, Fantasy, Kids, Slice of Life  
4294                       Comedy, School, Shounen  
1061        Comedy, School, Shounen, Slice of Life  
2850                 Comedy, School, Slice of Life  
